## Importación de Librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import gc
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import time
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.externals import joblib

## Lectura del Dataset

In [2]:
Consolidado=pd.read_csv('Dataset_consolidado.csv', error_bad_lines=False,warn_bad_lines=False,low_memory=False)

#### Muestra del dataset

In [3]:
Consolidado.head()

,seg_str_EMPRENDEDOR,seg_str_OTRO,seg_str_PERSONAL,seg_str_PERSONAL PLUS,seg_str_PREFERENCIAL,ocupacion_1,ocupacion_2,ocupacion_3,ocupacion_4,ocupacion_5,...,ingreso_rango_d. (3.3 4.4MM],ingreso_rango_e. (4.4 5.5MM],ingreso_rango_f. (5.5 6.6MM],ingreso_rango_g. (6.6 7.6MM],ingreso_rango_h. (7.6 8.7MM],ingreso_rango_i. (8.7 Inf),fecha,hora,valor_trx,etiqueta
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,20171009.0,90609.0,15100.91,1.0
1,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,20171013.0,101444.0,157299.02,2.0
2,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,20170825.0,175028.0,371718.55,2.0
3,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,20180124.0,104113.0,382742.66,2.0
4,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,20180325.0,153255.0,125253.38,2.0


#### Asignación de variable predictora y variable a predecir

In [4]:
y=Consolidado.etiqueta

In [5]:
X=Consolidado.iloc[:, Consolidado.columns != 'etiqueta']

#### liberación de memoria

In [6]:
del Consolidado
gc.collect()

#### Partición de los datos en entrenamiento y prueba

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.996,stratify=y)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.25,stratify=y_train)

In [11]:
X_train2.shape

(9591, 106)

In [12]:
X_test2.shape

(3197, 106)

## Definición del Modelo
Al dataset se le incorpora una reducción de dimensionalidad, luego se implementa el modelo SVC para clasificación multiclase. Finalmente se aplica GridSearchCV, el cual basado en validación cruzada y el score balanced_accuracy encuentra el mejor modelo dado por los diferentes hiperparametros.

Se elige balanced_accuracy como medición del score para modelos multiclase desbalanceados. 

In [14]:
parametros = {"pca__n_components": [2,3,5]}


pipeline = Pipeline(steps=[('pca', PCA()),
                           ('svc', SVC())])

score='balanced_accuracy'

grid_search_LP=GridSearchCV(pipeline, param_grid=parametros,cv=10,scoring=score,n_jobs=-1)

#### Ajuste del modelo

In [15]:
tiempo_inicio=time.clock()
grid_search_LP.fit(X_train2,y_train2)
tiempo_final=time.clock()
print("tiempo de ejecucion : "+str(int(tiempo_final-tiempo_inicio))+" segundos")

C:\Users\lssro\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.
C:\Users\lssro\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


tiempo de ejecucion : 171 segundos


C:\Users\lssro\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


#### Mejores parámetros encontrados 
con los que se construye el modelo

In [16]:
grid_search_LP.best_params_

{'pca__n_components': 2}

#### Selección del mejor modelo
Según los hiperparámetros encontrados

In [17]:
grid_search_LP.best_estimator_

Pipeline(memory=None,
     steps=[('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('svc', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

#### Score del mejor modelo

In [19]:
grid_search_LP.best_score_

0.3333333333333333

#### Matriz de confunsión
En la cual se analiza el desempeño del modelo según el número de predicciones de cada clase dadas por: VP, VN, FP, FN

In [22]:
confusion_matrix(y_test2, grid_search_LP.best_estimator_.predict(X_test2))

array([[   0, 1450,    0],
       [   0, 1711,    0],
       [   0,   36,    0]], dtype=int64)